# Import relevant libraries

In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.grid.grid_search import H2OGridSearch

In [2]:
# import encoders to deal with hccv's
from category_encoders import *

In [3]:
h2o.remove_all # clear h2o data

<function h2o.h2o.remove_all(retained=None)>

In [4]:
# Set directories
print(os.getcwd())
dirRawData = "../input/"
dirPData   = "../PData/"
dirPOutput = "../POutput/"

/home/jovyan/Projects/Fraud/PCode


# load data pickle from reading data notebook

In [5]:
f_name = dirPData + '01_df_250k.pickle'

with (open(f_name, "rb")) as f:
    dict_ = pickle.load(f)

df_train = dict_['df_train']
df_test  = dict_['df_test']

del f_name, dict_

f_name = dirPData + '01_vars.pickle'

with open(f_name, "rb") as f:
    dict_ = pickle.load(f)

vars_ind_numeric     = dict_['vars_ind_numeric']
vars_ind_hccv        = dict_['vars_ind_hccv']
vars_ind_categorical = dict_['vars_ind_categorical']
vars_notToUse        = dict_['vars_notToUse']
var_dep              = dict_['var_dep']

del f_name, dict_

In [6]:
# remove variables which remains constant
vars_ind_categorical.remove('c02')
vars_ind_numeric.remove('e16')

In [8]:
## constant variables added to notTouse 
vars_notToUse.append('c02')
vars_notToUse.append('e16')

In [9]:
# combine categorical and numeric variables 
vars_ind = vars_ind_categorical + vars_ind_numeric

In [68]:
var_dep

['target']

In [10]:
# we tried with and without standardising before splines but doing so before seems to worsen auc
# standardisation for train set

for var in vars_ind_numeric:
    x = df_train[var]
    x -= np.mean(x, axis=0)
    x /= np.sqrt(np.mean(x ** 2, axis=0))
    df_train[var] = x

In [11]:
# standardisation for train set

for var in vars_ind_numeric:
    x = df_test[var]
    x -= np.mean(x, axis=0)
    x /= np.sqrt(np.mean(x ** 2, axis=0))
    df_test[var] = x

In [12]:
# spline numeric variables which have more than 8 unique values, 
# we exerimented with different cutoff values apart from 8 
#but this didn't make much of a difference to the list of variables selected to spline

vars_ind_tospline = df_train[vars_ind_numeric].columns[(df_train[vars_ind_numeric].nunique() > 8)].tolist()

In [13]:
# adding splines to the numeric variables we randomly chose
# some numeric variables may have non-linear effects so we spline them
# the lecturer kept original variables in dataset so we thought to do the same
def fn_tosplines(varname,x):
    ptiles = np.percentile(x, [10, 20, 40, 60, 80, 90])
    ptiles = np.unique(ptiles)
    df_ptiles = pd.DataFrame({var: x})
    for idx, ptile in enumerate(ptiles):
        df_ptiles[var + '_' + str(idx)] = np.maximum(0, x - ptiles[idx])
    return(df_ptiles)

In [14]:
# create splines for train dataset
for var in vars_ind_tospline:
    df_ptiles = fn_tosplines(var,df_train[var])
    df_train.drop(columns=[var], inplace=True)
    vars_ind_numeric.remove(var)
    df_train = pd.concat([df_train, df_ptiles], axis=1, sort=False)
    vars_ind_numeric.extend(df_ptiles.columns.tolist())

In [15]:
# create splines for test dataset
for var in vars_ind_tospline:
    df1_ptiles = fn_tosplines(var,df_test[var])
    df_test.drop(columns=[var], inplace=True)
    vars_ind_numeric.remove(var)
    df_test = pd.concat([df_test, df1_ptiles], axis=1, sort=False)
    vars_ind_numeric.extend(df1_ptiles.columns.tolist())

In [16]:
len(vars_ind_numeric)

169

In [17]:
# update variable list
vars_ind = vars_ind_categorical + vars_ind_numeric

In [18]:
#set y variable for train set
y_train = df_train[var_dep].values.ravel()

In [19]:
#target encoder
# we tried target encoder but since we dont have y values for the test set we didn't us this

#enc = TargetEncoder(cols=['e17','e18','e19','f10'])

#df_encoded = enc.fit_transform(df_train, y_train)

In [20]:
#binary encoder
# we tried out binary encoder and it seemed to work well while not taking up much space in our dataframes
enc = BinaryEncoder(cols=['e17','e18','e19','f10'])

enc.fit(df_train[vars_ind_hccv])
df_encoded = enc.transform(df_train[vars_ind_hccv])
df_encoded[vars_ind_hccv] = df_train[vars_ind_hccv]
df_encoded.head()

#df_encoded = enc.fit_transform(df_train, y_train)

/opt/conda/lib/python3.9/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,e17_0,e17_1,e17_2,e17_3,e17_4,e17_5,e17_6,e17_7,e18_0,e18_1,...,f10_6,f10_7,f10_8,f10_9,f10_10,f10_11,e17,e18,e19,f10
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,38CF0,D556D,D2AFA,CHO
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,38CF0,D556D,D2AFA,AIR
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,1,38CF0,D556D,D2AFA,CHE
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,B6A15,E7317,CE2BB,CPH
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,1,B6A15,E7317,CE2BB,CUO


In [21]:
# drop encoded columns from df to avoid duplicates
df_encoded=df_encoded.drop(['e17', 'e18','e19','f10'], axis=1)

In [22]:
# extend categorical feature names with those resulting from encoding
vars_ind_categorical.extend(df_encoded.columns.tolist())

In [23]:
# concatenate original training df with encoded one
df_train = pd.concat([df_train, df_encoded], axis=1, sort=False)

In [24]:
# drop original variables from dataset as we have replaced them with encoded ones
df_train=df_train.drop(['e17', 'e18','e19','f10'], axis=1)

In [25]:
# binary encoder for test data
# same process as above for train data
enc = BinaryEncoder(cols=['e17','e18','e19','f10'])

enc.fit(df_test[vars_ind_hccv])
df_encoded = enc.transform(df_test[vars_ind_hccv])
df_encoded[vars_ind_hccv] = df_test[vars_ind_hccv]
df_encoded.head()


/opt/conda/lib/python3.9/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,e17_0,e17_1,e17_2,e17_3,e17_4,e17_5,e17_6,e17_7,e18_0,e18_1,...,f10_6,f10_7,f10_8,f10_9,f10_10,f10_11,e17,e18,e19,f10
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,920BD,11250,ECD18,CKG
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,F33BC,5614E,D9556,AIR
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,1,F33BC,5614E,D9556,AEL
3,0,0,0,0,0,0,1,1,0,0,...,0,0,0,1,0,0,861C8,9076B,463D0,CUO
4,0,0,0,0,0,0,1,1,0,0,...,0,0,0,1,0,1,861C8,9076B,463D0,AWV


In [26]:
# drop encoded columns from df to avoid duplicates
df_encoded=df_encoded.drop(['e17', 'e18','e19','f10'], axis=1)

In [27]:
# extend categorical feature names with those resulting from encoding
df_test = pd.concat([df_test, df_encoded], axis=1, sort=False)

In [28]:
# concatenate original training df with encoded one
df_test=df_test.drop(['e17', 'e18','e19','f10'], axis=1)

In [29]:
# we remove original variables that have been encoded from the list 
vars_ind_categorical.remove('e17')
vars_ind_categorical.remove('e18')
vars_ind_categorical.remove('e19')
vars_ind_categorical.remove('f10')


In [30]:
# these spline does not appear in test set when standardising earlier so we remove them from the list
vars_ind_numeric.remove('e09_4') 
vars_ind_numeric.remove('f01_4')

In [31]:
# update variable list
vars_ind = vars_ind_categorical + vars_ind_numeric

In [32]:
# add features engineered through division of numerics for train set
# we randomly chose some features to divide which did not have zeros to avoid values of infinity
# introduction of these features didn't seem to improve our auc
df_train['x0'] = df_train['e04']/df_train['e06']
df_train['x1'] = df_train['f13']/df_train['f02']
df_train['x2'] = df_train['e09']/df_train['f11']

# manually add interactions as the interaction_pairs argument didn't work in h2o
# we randomly chose some interactions between some variables for training set
# introduction of these features didn't seem to improve our auc
df_train['x3'] = df_train['e04']*df_train['e06']
df_train['x4'] = df_train['f13']*df_train['f02']
df_train['x5'] = df_train['e09']*df_train['f11']

In [33]:
# same as above for test set
df_test['x0'] = df_test['e04']/df_test['e06']
df_test['x1'] = df_test['f13']/df_test['f02']
df_test['x2'] = df_test['e09']/df_test['f11']

# same as above for test set
df_test['x3'] = df_test['e04']*df_test['e06']
df_test['x4'] = df_test['f13']*df_test['f02']
df_test['x5'] = df_test['e09']*df_test['f11']

In [34]:
# add new vars to vars_ind index
vars_ind.append('x0')
vars_ind.append('x1')
vars_ind.append('x2')
vars_ind.append('x3')
vars_ind.append('x4')
vars_ind.append('x5')

In [35]:
#check length of variables list
len(vars_ind)

261

In [36]:
# make sure no variables are repeated so we use set and then pass to list again
myset=set(vars_ind)

print(len(myset))

vars_ind=list(myset)


193


In [37]:
var_dep

['target']

In [38]:
df_train

,target,a01,a02,a03,a10,a12,a13,a16,a17,a18,...,f10_8,f10_9,f10_10,f10_11,x0,x1,x2,x3,x4,x5
0,1,D,C,H,Z,C,A,Z,Z,F,...,0,0,0,1,0.192403,-1.938869,-1.074925,0.216317,-3.662846,-1.238058
1,1,D,C,H,Z,C,A,Z,Z,F,...,0,0,1,0,0.192403,-0.491062,-0.875122,0.216317,-0.277127,-1.520726
2,0,D,C,H,Z,C,A,Z,Z,F,...,0,0,1,1,0.192403,-3.378403,-1.392959,0.216317,-0.380800,-0.955391
3,1,B,H,H,N,A,A,Y,N,F,...,0,1,0,0,-0.494890,-31.171417,-2.785252,-0.556401,-0.018121,-0.024136
4,0,B,H,H,N,A,A,Y,N,F,...,0,1,0,1,-0.494890,0.615723,-0.313070,-0.556401,0.221019,-0.214726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,1,B,A,H,N,A,A,Y,N,F,...,0,1,1,0,1.239706,0.942550,-0.875122,1.393792,0.144381,-1.520726
249996,0,B,A,B,N,A,A,N,N,F,...,0,1,0,1,-0.986678,0.240482,-1.392959,-2.682268,0.565889,-0.955391
249997,1,B,A,B,N,A,A,N,N,F,...,0,1,0,1,-0.986678,0.278152,-1.392959,-2.682268,0.489251,-0.955391
249998,0,B,A,B,N,A,A,N,N,F,...,0,1,0,1,-0.986678,-0.777890,-1.392959,-2.682268,-0.174943,-0.955391


In [39]:
df_all=df_train.copy()

In [48]:
# for clarity later on set df_all equal to complete training set
df_all=df_train.copy()

In [49]:
# sample from df_all and df_test for faster model computations
df_all=df_all.sample(10000, random_state=2021)

df_test_50=df_test.sample(10000, random_state=2021)

In [50]:
# add folds to randomly distinguish between train and validation sets
rng = np.random.RandomState(2018)
fold = rng.randint(0, 10, df_all.shape[0]) # takes values 0-9 inclusive
df_all['fold'] = fold

In [51]:
# set index according to fold value
idx_train  = df_all['fold'].isin(range(9))
idx_val    = df_all['fold'].isin([9, 10])
idx_design = df_all['fold'].isin(range(10))

print("number of train examples",    np.sum(idx_train == 1))
print("number of val examples",      np.sum(idx_val == 1))
print("number of design examples",   np.sum(idx_design == 1))

number of train examples 9046
number of val examples 954
number of design examples 10000


In [52]:
#initialise h2o connection
h2o.init(ip='localhost', 
         port=54321, 
         https=False,enable_assertions = False)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 day 2 hours 58 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.2
H2O_cluster_version_age:,2 months and 16 days
H2O_cluster_name:,H2O_from_python_jovyan_w8onbs
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.554 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [53]:
#pass df_all to h2o

h2o_df_all = h2o.H2OFrame(df_all[vars_ind + var_dep + ['fold']],
                          destination_frame = 'df_all')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [54]:
#pass index to h2o

idx_h2o_train  = h2o.H2OFrame(idx_train.astype('int').values,  
                              destination_frame = 'idx_h2o_train')
idx_h2o_val    = h2o.H2OFrame(idx_val.astype('int').values  ,  
                              destination_frame = 'idx_h2o_val')
idx_h2o_design = h2o.H2OFrame(idx_design.astype('int').values, 
                              destination_frame = 'idx_h2o_design')


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [55]:
#set manually

h2o_df_design = h2o_df_all[idx_h2o_design, :]
h2o_df_train  = h2o_df_all[idx_h2o_train, :]
h2o_df_val    = h2o_df_all[idx_h2o_val, :]

In [61]:
#pass test df to h2o
h2o_df_test=h2o.H2OFrame(df_test[vars_ind],
                          destination_frame = 'df_test')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [124]:
# Set up the hyper parameters to search over
# We will  search over 6 values for alpha  

alpha_opts = [0.1,0.2,0.4,0.6,0.75,0.99]

#alpha_opts = np.arange(0, 1, 0.).tolist()

hyper_parameters = {"alpha":alpha_opts}

In [125]:
criteria = {# The default strategy, "Cartesian", covers the entire space of h-p combinations. 
            "strategy": "RandomDiscrete", 
            "max_models": 6,
            "stopping_metric": "AUTO",
            "seed": 2021}

grid = H2OGridSearch(H2OGeneralizedLinearEstimator(family="binomial",
                                                   nfolds = 10,
                                                   lambda_search=True),
                     hyper_params=hyper_parameters,
                     grid_id='g5',
                     search_criteria=criteria
                    )

grid.train(y = var_dep,
           x = vars_ind, 
           training_frame=h2o_df_all[idx_h2o_train, :],
           validation_frame=h2o_df_all[idx_h2o_val,:] 
           )

glm Grid Build progress: |████████████████████████████████████████████████| 100%


In [126]:
grid = grid.get_grid(sort_by='AUC', decreasing=True)
grid

       alpha    model_ids                 auc
0      [0.4]   g5_model_3  0.7271663385975469
1     [0.99]   g5_model_2  0.7271206364410034
2      [0.6]   g5_model_4  0.7268291929144829
3     [0.75]   g5_model_6  0.7266378990963563
4      [0.2]   g5_model_5  0.7262986067356209
5      [0.1]   g5_model_1  0.7259440096866927
6     [0.99]   g5_model_8  0.7155528109982147
7     [0.75]  g5_model_12  0.7141706814495349
8      [0.6]  g5_model_10  0.7128647189891542
9      [0.2]  g5_model_11  0.7103943484998237
10     [0.4]   g5_model_9  0.7101319924796978
11     [0.1]   g5_model_7  0.7074168411103205


In [113]:
df_all

,target,a01,a02,a03,a10,a12,a13,a16,a17,a18,...,f10_9,f10_10,f10_11,x0,x1,x2,x3,x4,x5,fold
211451,0,B,C,H,N,A,A,N,N,F,...,1,1,0,0.941815,0.729448,-1.434972,0.872459,0.215457,-0.591435,6
96383,1,B,A,D,N,A,A,Y,N,F,...,0,0,0,-0.018496,0.008338,-3.411868,-0.050282,0.022742,-0.390056,2
55772,0,B,C,H,N,A,A,Y,N,F,...,1,0,1,-0.331249,0.246036,-1.392959,-0.372421,0.553116,-0.955391,9
205550,0,B,A,E,N,A,A,Y,N,F,...,1,0,1,0.846968,0.301789,-1.392959,0.952239,0.450933,-0.955391,5
157239,1,B,A,H,N,A,A,Y,N,F,...,1,1,0,0.545204,0.012548,-0.305297,0.505054,0.015113,-0.578949,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188056,0,B,C,H,N,A,A,N,N,F,...,1,0,1,-95.187234,-0.074988,-3.221876,-0.015606,-0.002529,-0.507713,3
123186,1,B,B,H,N,A,A,N,N,F,...,1,0,1,-1.149455,0.225231,2.019488,-1.292323,0.604208,1.385110,4
157104,1,B,A,B,N,A,A,Y,N,F,...,0,0,1,0.364926,-0.027805,-0.444618,0.338052,-0.006820,-0.151196,7
40243,0,B,C,H,N,A,A,N,N,F,...,0,1,0,0.761537,0.010838,0.158408,0.705457,0.017497,0.182449,4


In [70]:
# again with more narrower alpha range

alpha_opts = [0.1,0.2,0.4,0.99]

#alpha_opts = np.arange(0, 1, 0.).tolist()

hyper_parameters = {"alpha":alpha_opts}

criteria = {# The default strategy, "Cartesian", covers the entire space of h-p combinations. 
            "strategy": "RandomDiscrete", 
            "max_models": 2,
            "stopping_metric": "AUTO",
            "seed": 2021}

grid = H2OGridSearch(H2OGeneralizedLinearEstimator(family="binomial",
                                                   nfolds = 10,
                                                   lambda_search=True),
                     hyper_params=hyper_parameters,
                     grid_id='g6',
                     search_criteria=criteria
                    )

grid.train(y = var_dep,
           x = vars_ind, 
           training_frame=h2o_df_all[idx_h2o_train, :],
           validation_frame=h2o_df_all[idx_h2o_val,:] 
           )

glm Grid Build progress: |████████████████████████████████████████████████| 100%


In [71]:
grid1 = grid.get_grid(sort_by='AUC', decreasing=True)
grid1

     alpha   model_ids                 auc
0    [0.4]  g6_model_1  0.7296772366190788
1    [0.2]  g6_model_2  0.7296077038833862


In [56]:
# train penalised logistic regression model on training set and pass validation frame as well. 
# we experimented with grid search first and got different results for optimal value
# of alpha depending on max models, runtime and nfolds, eventually we chose 0.4 and ran on 250k train set
# we set nlambdas to 20 as this was default
# we choose 10 nfolds for computation time
#interaction_pairs = [('a01', 'a02')] - did not work



model=H2OGeneralizedLinearEstimator(alpha=0.4, 
                                    lambda_search=True,
                                    lambda_min_ratio=1e-8,
                                    nlambdas=20,
                                    nfolds=10,
                                    early_stopping=True,
                                    family='binomial',
                                    #interaction_pairs=interaction_pairs, - did not work
                                    # we already standardised above
                                    standardize=False,
                                    seed=2020)

model.train(x=vars_ind, 
            y='target',
            training_frame=h2o_df_all[idx_h2o_train, :],
           validation_frame=h2o_df_all[idx_h2o_val,:])

# Predict the model on train and val
model_pred_train = model.predict(h2o_df_all[idx_h2o_train, :])
model_pred_val   = model.predict(h2o_df_all[idx_h2o_val, :])

model_pred_train = model_pred_train.as_data_frame().values.ravel()
model_pred_val   = model_pred_val.as_data_frame().values.ravel()

/opt/conda/lib/python3.9/site-packages/h2o/estimators/estimator_base.py:200: RuntimeWarning: Dropping bad and constant columns: [e19_0, e17_0, f10_0, f06_2, e18_0, a20]
  warnings.warn(mesg["message"], RuntimeWarning)


glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


In [60]:
model

Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_model_python_1626346102456_11562


GLM Model: summary


,,family,link,regularization,lambda_search,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,binomial,logit,"Elastic Net (alpha = 0.4, lambda = 6.193E-4 )","nlambda = 20, lambda.max = 0.2081, lambda.min = 6.193E-4, lambda.1...",368,213,16,py_1_sid_8c62




ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.20372932263675242
RMSE: 0.45136384728592566
LogLoss: 0.5925776267276079
Null degrees of freedom: 9045
Residual degrees of freedom: 8832
Null deviance: 12537.298558247585
Residual deviance: 10720.914422755885
AIC: 11148.914422755885
AUC: 0.7480757535534752
AUCPR: 0.7416168129386597
Gini: 0.49615150710695044

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3733317967604742: 


,,0,1,Error,Rate
0,0,2091.0,2516.0,0.5461,(2516.0/4607.0)
1,1,634.0,3805.0,0.1428,(634.0/4439.0)
2,Total,2725.0,6321.0,0.3482,(3150.0/9046.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...


See the whole table with table.as_data_frame()

Scoring History: 


,,timestamp,duration,iteration,lambda,predictors,deviance_train,deviance_test,deviance_xval,deviance_se,...,training_pr_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_r2,validation_auc,validation_pr_auc,validation_lift,validation_classification_error
0,,2021-07-16 13:48:03,0.000 sec,1,.21E0,1,1.385949,1.387340,1.386286,0.000444,...,,,,,,,,,,
1,,2021-07-16 13:48:03,0.337 sec,3,.79E-1,9,1.370315,1.372231,1.374259,0.001130,...,,,,,,,,,,
2,,2021-07-16 13:48:04,0.533 sec,5,.3E-1,34,1.321388,1.324715,1.332127,0.004095,...,,,,,,,,,,
3,,2021-07-16 13:48:04,0.728 sec,7,.11E-1,57,1.271872,1.287063,1.285836,0.008041,...,,,,,,,,,,
4,,2021-07-16 13:48:04,0.927 sec,9,.43E-2,105,1.233255,1.273289,1.254388,0.010230,...,,,,,,,,,,
5,,2021-07-16 13:48:04,1.234 sec,12,.16E-2,159,1.203979,1.263687,1.235311,0.011709,...,,,,,,,,,,
6,,2021-07-16 13:48:05,1.655 sec,16,.62E-3,214,1.185155,1.257708,1.227318,0.012595,...,,,,,,,,,,
7,,2021-07-16 13:48:05,1.981 sec,19,.23E-3,264,1.174760,1.259766,1.228215,0.012893,...,,,,,,,,,,
8,,2021-07-16 13:48:05,2.205 sec,21,.89E-4,309,1.169096,1.264696,1.231877,0.012998,...,,,,,,,,,,
9,,2021-07-16 13:48:06,2.432 sec,23,.34E-4,321,1.166208,1.270625,1.236070,0.012886,...,0.741617,1.970665,0.34822,0.468421,0.628854,0.122016,0.701793,0.693683,1.374074,0.362683



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,f13_0,1.708895,1.000000,0.062259
1,f09.F,1.577223,0.922949,0.057462
2,f13,1.000692,0.585579,0.036458
3,f13_1,0.872666,0.510661,0.031793
4,f11,0.838468,0.490649,0.030547
5,f03.F,0.683011,0.399680,0.024884
6,f11_1,0.628390,0.367717,0.022894
7,f11_0,0.619616,0.362583,0.022574
8,e20.54CD2,0.581102,0.340046,0.021171
9,e11.A,0.500708,0.293001,0.018242



See the whole table with table.as_data_frame()


In [62]:
model_pred_test=model.predict(h2o_df_test).as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%


/opt/conda/lib/python3.9/site-packages/h2o/job.py:72: UserWarning: Test/Validation dataset column 'e20' has levels not trained on: ["30146", "3D6DA", "40897", "B2E8B", "BE271", "DCD42"]
  warnings.warn(w)
/opt/conda/lib/python3.9/site-packages/h2o/job.py:72: UserWarning: Test/Validation dataset column 'f09' has levels not trained on: ["L"]
  warnings.warn(w)
/opt/conda/lib/python3.9/site-packages/h2o/job.py:72: UserWarning: Test/Validation dataset column 'e24' has levels not trained on: ["J", "M", "O", "Z"]
  warnings.warn(w)
/opt/conda/lib/python3.9/site-packages/h2o/job.py:72: UserWarning: Test/Validation dataset column 'e03' has levels not trained on: ["J", "O", "Z"]
  warnings.warn(w)
/opt/conda/lib/python3.9/site-packages/h2o/job.py:72: UserWarning: Test/Validation dataset column 'e13' has levels not trained on: ["H", "J", "K", "M", "P", "Q", "S"]
  warnings.warn(w)
/opt/conda/lib/python3.9/site-packages/h2o/job.py:72: UserWarning: Test/Validation dataset column 'f03' has levels n

In [66]:
model_pred_test

,predict,p0,p1
0,1,0.436217,0.563783
1,1,0.088731,0.911269
2,1,0.496274,0.503726
3,1,0.297664,0.702336
4,1,0.371687,0.628313
...,...,...,...
296685,1,0.453223,0.546777
296686,0,0.649669,0.350331
296687,1,0.351903,0.648097
296688,0,0.663141,0.336859


In [63]:
# add test predictions and uniqie id to dataframe

df_sub_14 = pd.DataFrame({
        "unique_id": df_test["unique_id"],
        "Predicted": model_pred_test['p1']
    })

In [67]:
df_sub_14.to_csv('../POutput/df_sub_part3.csv', index=False)

In [65]:
df_sub_14

,unique_id,Predicted
0,6,0.563783
1,16,0.911269
2,17,0.503726
3,18,0.702336
4,19,0.628313
...,...,...
296685,2265630,0.546777
296686,2265631,0.350331
296687,2265632,0.648097
296688,2265637,0.336859


In [ ]:
# get whole 250k train set back and set up train,val,design and test frames and index - we didn't run this section becuase of computation time

In [43]:
# for clarity later on set df_all equal to complete training set
df_all=df_train.copy()


# add folds to randomly distinguish between train and validation sets
rng = np.random.RandomState(2018)
fold = rng.randint(0, 10, df_all.shape[0]) # takes values 0-9 inclusive
df_all['fold'] = fold

# set index according to fold value
idx_train  = df_all['fold'].isin(range(9))
idx_val    = df_all['fold'].isin([9, 10])
idx_design = df_all['fold'].isin(range(10))

print("number of train examples",    np.sum(idx_train == 1))
print("number of val examples",      np.sum(idx_val == 1))
print("number of design examples",   np.sum(idx_design == 1))

number of train examples 223921
number of val examples 24984
number of design examples 248905


In [68]:
#pass df_all to h2o

h2o_df_all = h2o.H2OFrame(df_all[vars_ind + var_dep + ['fold']],
                          destination_frame = 'df_all')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
#pass index to h2o

idx_h2o_train  = h2o.H2OFrame(idx_train.astype('int').values,  
                              destination_frame = 'idx_h2o_train')
idx_h2o_val    = h2o.H2OFrame(idx_val.astype('int').values  ,  
                              destination_frame = 'idx_h2o_val')
idx_h2o_design = h2o.H2OFrame(idx_design.astype('int').values, 
                              destination_frame = 'idx_h2o_design')

In [ ]:
#set manually

h2o_df_design = h2o_df_all[idx_h2o_design, :]
h2o_df_train  = h2o_df_all[idx_h2o_train, :]
h2o_df_val    = h2o_df_all[idx_h2o_val, :]

#pass test df to h2o
h2o_df_test=h2o.H2OFrame(df_test_50[vars_ind],
                          destination_frame = 'df_test')

In [201]:
model_pred_train = model.predict(h2o_df_all[idx_h2o_train, :]).as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%


In [169]:
#df_test.loc[:,'Predicted']=pred

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [ ]:
# run model again for large dataset

##model=H2OGeneralizedLinearEstimator(alpha=0.4, 
                                    lambda_search=True,
                                    lambda_min_ratio=1e-8,
                                    nlambdas=20,
                                    nfolds=10,
                                    early_stopping=True,
                                    family='binomial',
                                    #interaction_pairs=interaction_pairs, - did not work
                                    # we already standardised above
                                    standardize=False,
                                    seed=2020)

model.train(x=vars_ind, 
            y='target',
            training_frame=h2o_df_all[idx_h2o_train, :],
           validation_frame=h2o_df_all[idx_h2o_val,:])

# Predict the model on train and val
model_pred_train = model.predict(h2o_df_all[idx_h2o_train, :])
model_pred_val   = model.predict(h2o_df_all[idx_h2o_val, :])

model_pred_train = model_pred_train.as_data_frame().values.ravel()
model_pred_val   = model_pred_val.as_data_frame().values.ravel()